In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sqlite3
from urllib.request import urlopen
import json
import requests

import plotly.express as px

In [2]:
df = pd.read_csv('../StartingWithToday/data/non-profit-orgs.csv')

Access the data that was web-scrapped and stored in the database.

In [3]:
df.shape

(1000, 10)

In [4]:
df.head(3)

,org_url,name,location,website,about,services,org_type,lat,lng,zip
0,https://www.idealist.org/en/nonprofit/54bb0a44...,Maryland/DC Odyssey of the Mind,"Maryland Ave, Bethesda, MD 20816, USA",mdodyssey.org?utm_medium=referral&amp;utm_sour...,No value,No value,No value,38.9446992,-77.1210422,20816
1,https://www.idealist.org/en/nonprofit/3ccbd3d6...,Center for the Study of Social Policy,"1575 Eye Street, NW - Suite 500, Washington, D...",www.cssp.org?utm_medium=referral&amp;utm_sourc...,Organization Overview</strong></p><p>The Cente...,Children &amp; Youth,Nonprofit,38.9016662,-77.03546519999999,20005
2,https://www.idealist.org/en/nonprofit/71cf8af9...,HIAS,"1300 Spring Street, Silver Spring, MD 20910, U...",www.hias.org?utm_medium=referral&amp;utm_sourc...,No value,No value,No value,38.9989623,-77.0319777,20910


In [5]:
df.isnull().sum()

org_url     0
name        0
location    2
website     0
about       0
services    0
org_type    0
lat         0
lng         0
zip         2
dtype: int64

In [6]:
df.dtypes

org_url     object
name        object
location    object
website     object
about       object
services    object
org_type    object
lat         object
lng         object
zip         object
dtype: object

In [ ]:
df_svcs = pd.DataFrame(df.groupby(['services','zip'])['org_name'].count())
df_svcs.reset_index(inplace=True)

In [ ]:
df_svcs = df_svcs[df_svcs['services'] != '']

In [ ]:
df_svcs['services'] = [service.replace('&amp;','&') for service in df_svcs['services']]

In [ ]:
df_svcs.rename(columns={'org_name':'nbr_orgs'},inplace=True)

In [ ]:
df_svcs = pd.pivot(df_svcs, index=['zip'], columns=['services'], values='nbr_orgs')

In [ ]:
df_svcs.reset_index(inplace=True)

In [ ]:
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/dc_district_of_columbia_zip_codes_geo.min.json') as response:
    zips = json.load(response)

In [ ]:
fig = px.choropleth(df_svcs,
                    geojson=zips, 
                    locations='zip', 
                    color='Infancia & Juventud',
                    color_continuous_scale="Viridis",
                    range_color=(1,50),
                    featureidkey="properties.ZCTA5CE10",
                    scope="usa",
                    center = {"lat": 38.927216, "lon": -77.1034053})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                 title_text = 'Number of non-profits in DC')